In [8]:
# Import essential modules
import os
import pandas as pd
import FrozenPy.functions as fp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

In [9]:
def detect_freezing(Threshold_df, threshold = 10):

    """
    Detects freezing using a threshold from raw motion (MedPC Threshold) data. This function
    loops through the array one row at a time and determines if the values are < 'threshold' for
    at least one second. A new array of 1s and 0s are created denoting freezing and not freezing,
    respectively.

    Note: This code will need to be updated with inputs defining Fs and freezing (ex. immobile for 
    < 1 sec or more/less) to be broadly useful outside the Maren Lab. 

    Parameters
    ----------
    Threshold_df : pandas dataframe
        dataframe generated from readRaw()
    threshold : int, optional
        desired threshold, 10 by default

    Returns
    -------
    Behav_df : pandas dataframe
        dataframe containing both freezing (key: 'Freezing') and raw motion (key: 'Threshold') data.
        Freezing data is an array of 1s and 0s denoting freezing and not-freezing.
    """

    Threshold_values = Threshold_df.values
    rows = np.size(Threshold_values, 0)
    columns = np.size(Threshold_values, 1)

    Freezing_np = np.zeros((rows, columns))
    for c in range(columns):
        for r in range(5, rows):
            data2check = Threshold_values[r-5:r, c] # 1 second of data to check

            if all(n <= threshold  for n in data2check):
                Freezing_np[r, c] = 1

    Freezing_df = pd.DataFrame(Freezing_np)
    Corrected_Freezing = _correct_freezing(Freezing_df)
    Corrected_Freezing.columns = Threshold_df.columns
    
    Behav_df = pd.concat([Threshold_df, Corrected_Freezing],keys=['Threshold', 'Freezing'])

    return Behav_df

def _correct_freezing(Freezing_df):

    """
    Corrects freezing detected by detectFreezing(), which only defines 
    freezing one sample at a time,and thus cannot account for freezing onset in
    which 1 second of freezing must be counted. For example, at freezing onset 
    5 samples (1 sec) must be below below threhsold values, but only detectFreezing()
    only defines 0.2 sec of freezing at time. So, this function looks for 
    freezing onset and changes that '1' to a '5' to account for this.

    Note: This code will also need to be updated with Fs to be used outside the Maren Lab.

    Parameters
    ----------
    Freezing_df : pandas dataframe
        dataframe generated from detectFreezing()

    Returns
    -------
    Corrected_Freezing_df : pandas dataframe
        dataframe containing 0s, 1s, and 5s denoting not-freezing, freezing, and freezing-onset.
    """

    # prep data
    Freezing_values = Freezing_df.values
    rows = np.size(Freezing_values,0)
    columns = np.size(Freezing_values,1)

    # correct freezing
    Freezing_final = np.zeros((rows,columns))
    for c in range(0,columns):

        for r in range(4,rows):
            previous = Freezing_values[r-1,c]
            current = Freezing_values[r,c]

            if current == 0:
                Freezing_final[r, c] = 0
            elif current == 1 and previous == 0:
                Freezing_final[r-4:r+1, c] = 1
            elif current == 1 and previous == 1:
                Freezing_final[r, c] = 1

    Corrected_Freezing_df = pd.DataFrame(Freezing_final)

    return Corrected_Freezing_df

In [10]:
# read .out files from data directory
data_dir = '\\Users\micstott\Documents\Python stuff\my-modules\FrozenPy\Examples\data'
fp.read_out(data_dir)

# change current directory to data directory
os.chdir(data_dir)

# Load Conditioning data
Cond_S1 = fp.read_rawcsv('Example_Conditioning_Sq1.raw.csv')
Cond_S2 = fp.read_rawcsv('Example_Conditioning_Sq2.raw.csv')
Cond_S3 = fp.read_rawcsv('Example_Conditioning_Sq3.raw.csv')

# Load Extinction data
Ext_S1 = fp.read_rawcsv('Example_Extinction_Sq1.raw.csv')
Ext_S2 = fp.read_rawcsv('Example_Extinction_Sq2.raw.csv')
Ext_S3 = fp.read_rawcsv('Example_Extinction_Sq3.raw.csv')

# Load Retrieval data
Ret_S1 = fp.read_rawcsv('Example_Retrieval_Sq1.raw.csv')
Ret_S2 = fp.read_rawcsv('Example_Retrieval_Sq2.raw.csv')
Ret_S3 = fp.read_rawcsv('Example_Retrieval_Sq3.raw.csv')

"Example_Conditioning_Sq1.raw.csv" already exists. Delete to rewrite file.
"Example_Conditioning_Sq2.raw.csv" already exists. Delete to rewrite file.
"Example_Conditioning_Sq3.raw.csv" already exists. Delete to rewrite file.
"Example_Extinction_Sq1.raw.csv" already exists. Delete to rewrite file.
"Example_Extinction_Sq2.raw.csv" already exists. Delete to rewrite file.
"Example_Extinction_Sq3.raw.csv" already exists. Delete to rewrite file.
"Example_Retrieval_Sq1.raw.csv" already exists. Delete to rewrite file.
"Example_Retrieval_Sq2.raw.csv" already exists. Delete to rewrite file.
"Example_Retrieval_Sq3.raw.csv" already exists. Delete to rewrite file.
Finished reading .out files
Dropped "DELETE" columns in Example_Conditioning_Sq1.raw.csv
Dropped "DELETE" columns in Example_Conditioning_Sq2.raw.csv
Dropped "DELETE" columns in Example_Conditioning_Sq3.raw.csv
Dropped "DELETE" columns in Example_Extinction_Sq1.raw.csv
Dropped "DELETE" columns in Example_Extinction_Sq2.raw.csv
Dropped "DE

In [11]:
#==============================
# Correct data
#==============================

# Freezing to Threshold
Cond_Sq1_df = fp.detect_freezing(Cond_S1)
Cond_Sq2_df = fp.detect_freezing(Cond_S2)
Cond_Sq3_df = fp.detect_freezing(Cond_S3)

# Concatenate all squad dataframes
Cond_df = pd.concat([Cond_Sq1_df, Cond_Sq2_df, Cond_Sq3_df], axis=1)
Cond_df.xs('Freezing').head()

,Rat7,Rat12,Rat17,Rat1,Rat13,Rat8,Rat2,Rat5,Rat20,Rat16,...,Rat21,Rat6,Rat11,Rat18,Rat3,Rat9,Rat14,Rat4,Rat19,Rat15
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#==============================
# Test data by changing _corrected_freezing
#==============================

# Freezing to Threshold
Cond_Sq1_df2 = detect_freezing(Cond_S1)
Cond_Sq2_df2 = detect_freezing(Cond_S2)
Cond_Sq3_df2 = detect_freezing(Cond_S3)

# Concatenate all squad dataframes
Cond_df2 = pd.concat([Cond_Sq1_df2, Cond_Sq2_df2, Cond_Sq3_df2], axis=1)
Cond_df2.xs('Freezing').mean()

Rat7     0.353464
Rat12    0.130419
Rat17    0.194146
Rat1     0.103001
Rat13    0.240089
Rat8     0.172286
Rat2     0.024454
Rat5     0.254539
Rat20    0.207114
Rat16    0.003705
Rat10    0.088181
Rat21    0.221564
Rat6     0.035939
Rat11    0.007040
Rat18    0.094479
Rat3     0.050760
Rat9     0.263060
Rat14    0.202297
Rat4     0.410152
Rat19    0.203409
Rat15    0.224528
dtype: float64

In [13]:
plt.figure()
plt.plot(Cond_df['Rat12'].xs('Freezing'))
plt.plot(Cond_df2['Rat12'].xs('Freezing'))